## Tape Structure: 
1. The tape consists of elements to sort followed by a swap flag (0 or 1) indicating if any swaps occurred during the pass.

## States:
1. *start*: Begins a new pass.

2. *compare*: Compares current and next elements.

3. *swap*: Swaps elements and sets the swap flag.

4. *check_flag*: Checks if another pass is needed.

## Transitions: 
Based on the current state and tape symbol, the machine moves the head, compares elements, swaps if necessary, and checks the swap flag to continue or halt.

In [4]:
class TuringMachine:
    def __init__(self, tape):
        self.tape = tape.copy() + ['0']  # Add swap flag at the end
        self.head = 0
        self.state = 'start'
        self.swap_flag_pos = len(self.tape) - 1
        self.history = []  # To record all steps
        self.step_counter = 0

    def log_action(self, action):
        self.step_counter += 1
        self.history.append(f"Step {self.step_counter}: {action}")

    def step(self):
        if self.state == 'halt':
            return

        current_symbol = self.tape[self.head] if self.head < len(self.tape) else None

        if self.state == 'start':
            action = f"START NEW PASS | Reset head to position 0"
            self.head = 0
            self.state = 'compare'
            self.log_action(action)

        elif self.state == 'compare':
            if self.head >= self.swap_flag_pos - 1:
                action = "Reached end of elements | Check swap flag"
                self.state = 'check_flag'
                self.log_action(action)
            else:
                next_symbol = self.tape[self.head + 1]
                if current_symbol > next_symbol:
                    action = f"COMPARE [{current_symbol}↔{next_symbol}] at positions {self.head}-{self.head+1} | Need swap"
                    self.state = 'swap'
                    self.log_action(action)
                else:
                    action = f"COMPARE [{current_symbol}↔{next_symbol}] at positions {self.head}-{self.head+1} | No swap"
                    self.head += 1
                    if self.head >= self.swap_flag_pos - 1:
                        self.state = 'check_flag'
                        action += " | Reached end"
                    self.log_action(action)

        elif self.state == 'swap':
            # Perform swap
            prev_val = self.tape[self.head]
            next_val = self.tape[self.head + 1]
            self.tape[self.head], self.tape[self.head + 1] = next_val, prev_val
            self.tape[self.swap_flag_pos] = '1'
            
            action = f"SWAP [{prev_val}⇄{next_val}] at positions {self.head}-{self.head+1} | "
            self.head += 1
            action += f"Moved to position {self.head} | Set swap flag"
            
            if self.head >= self.swap_flag_pos - 1:
                self.state = 'check_flag'
                action += " | Reached end"
            else:
                self.state = 'compare'
                
            self.log_action(action)

        elif self.state == 'check_flag':
            if self.tape[self.swap_flag_pos] == '1':
                action = "CHECK FLAG [1] | Swaps occurred | Reset flag and restart"
                self.tape[self.swap_flag_pos] = '0'
                self.state = 'start'
            else:
                action = "CHECK FLAG [0] | No swaps | Sorting complete"
                self.state = 'halt'
            self.log_action(action)

    def run(self):
        while self.state != 'halt':
            self.step()
        return self.tape[:-1], self.history

# Example usage with detailed logging
initial_elements = ['4', '7', '2', '1','10','29','12','9']
tm = TuringMachine(initial_elements)
sorted_tape, history = tm.run()

print("Initial tape:", initial_elements)
print("Sorted tape:", sorted_tape)
print("\nStep-by-step execution log:")
print("\n".join(history))

Initial tape: ['4', '7', '2', '1', '10', '29', '12', '9']
Sorted tape: ['1', '10', '12', '2', '29', '4', '7', '9']

Step-by-step execution log:
Step 1: START NEW PASS | Reset head to position 0
Step 2: COMPARE [4↔7] at positions 0-1 | No swap
Step 3: COMPARE [7↔2] at positions 1-2 | Need swap
Step 4: SWAP [7⇄2] at positions 1-2 | Moved to position 2 | Set swap flag
Step 5: COMPARE [7↔1] at positions 2-3 | Need swap
Step 6: SWAP [7⇄1] at positions 2-3 | Moved to position 3 | Set swap flag
Step 7: COMPARE [7↔10] at positions 3-4 | Need swap
Step 8: SWAP [7⇄10] at positions 3-4 | Moved to position 4 | Set swap flag
Step 9: COMPARE [7↔29] at positions 4-5 | Need swap
Step 10: SWAP [7⇄29] at positions 4-5 | Moved to position 5 | Set swap flag
Step 11: COMPARE [7↔12] at positions 5-6 | Need swap
Step 12: SWAP [7⇄12] at positions 5-6 | Moved to position 6 | Set swap flag
Step 13: COMPARE [7↔9] at positions 6-7 | No swap | Reached end
Step 14: CHECK FLAG [1] | Swaps occurred | Reset flag and r

## 2. Transition Table

| Current State | Read Symbol | Next State  | Write Symbol | Move Direction | Condition/Description      |
|--------------|------------|-------------|--------------|---------------|----------------------------|
| start        | *          | compare     | same         | R             | Begin new pass             |
| compare      | a          | compare     | a            | R             | a ≤ next element           |
| compare      | a          | swap        | a            | -             | a > next element           |
| swap         | a          | compare     | b            | R             | Swap a↔b, set flag         |
| compare      | ☐         | check_flag  | ☐           | L             | End of elements            |
| check_flag   | 0          | halt        | 0            | -             | No swaps → sorted          |
| check_flag   | 1          | start       | 0            | L             | Reset flag → new pass      |

### Where:
- `a, b` ∈ input alphabet (e.g., digits 0-9)
- `☐` = blank symbol (end marker)
- `R / L` = move right/left


Step | State       | Tape (Head Position)       | Action
-----|-------------|-----------------------------|---------------------------
1    | start       | [3̲,1,2,4,0]                 | Reset head
2    | compare     | [3̲,1,2,4,0]                 | Compare 3 > 1 → swap
3    | swap        | [1̲,3,2,4,1]                 | Swap completed
4    | compare     | [1,3̲,2,4,1]                 | Compare 3 > 2 → swap
5    | swap        | [1,2̲,3,4,1]                 | Swap completed
6    | compare     | [1,2,3̲,4,1]                 | Compare 3 < 4 → move
7    | check_flag  | [1,2,3,4,1̲]                | Flag=1 → new pass
8    | start       | [1̲,2,3,4,0]                 | Reset for next pass
9    | compare     | [1̲,2,3,4,0]                 | All elements sorted → halt